#### Import Data

In [119]:
# LIBRARIES
import pandas as pd
import numpy as np
from numpy import *

# IMPORT
df = pd.read_table("C:\\Users\\alina\\OneDrive\\Documents\\T2 2021\\SIT723\\FILES\\df2.txt")
df.isnull().sum()
#df = df[['Energy', 'Protein', 'Fat', 'Sat_Fat', 'Carbo_Hydrate', 'Sugar', 'Sodium', 'ADG']]
df2 = (df.dropna().reset_index()).drop('index', 1)
df2[:5]

# DATA TO FLOAT
df2["Energy"] = pd.to_numeric(df2["Energy"], downcast="float")
df2["Protein"] = pd.to_numeric(df2["Protein"], downcast="float")
df2["Fat"] = pd.to_numeric(df2["Fat"], downcast="float")
df2["Sat_Fat"] = pd.to_numeric(df2["Sat_Fat"], downcast="float")
df2["Carbo_Hydrate"] = pd.to_numeric(df2["Carbo_Hydrate"], downcast="float")
df2["Sugar"] = pd.to_numeric(df2["Sugar"], downcast="float")
df2["Sodium"] = pd.to_numeric(df2["Sodium"], downcast="float")

# DATA SHAPE
print(df2.shape)

# DEFINE X AND Y
x = (df2[['Source', 'Brand', 'Product name', 'Category', 'Energy', 'Protein', 'Fat', 'Sat_Fat', 'Carbo_Hydrate', 'Sugar', 'Sodium']])
y = (df2['ADG'])

# TRAIN/TEST SPLIT
from sklearn.model_selection import train_test_split
train_x0, test_x0, train_y, test_y = train_test_split(x, y, test_size=0.4, random_state=42, stratify=y)

train_x = train_x0[['Energy', 'Protein', 'Fat', 'Sat_Fat', 'Carbo_Hydrate', 'Sugar', 'Sodium']]
test_x = test_x0[['Energy', 'Protein', 'Fat', 'Sat_Fat', 'Carbo_Hydrate', 'Sugar', 'Sodium']]

print(np.unique(train_y, return_counts=True))
print(np.unique(test_y, return_counts=True))

(5860, 19)
(array([1, 2, 3, 7, 8], dtype=int64), array([1386, 1985,   58,   76,   11], dtype=int64))
(array([1, 2, 3, 7, 8], dtype=int64), array([ 924, 1323,   39,   50,    8], dtype=int64))


#### Random Forest classifier: Fit, Prediction, Accuracy, Classification Report

In [340]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest Classifier
clf=RandomForestClassifier(n_estimators=100, criterion="entropy",
                          min_samples_split=3, bootstrap=False, 
                           n_jobs=-3, random_state=0, 
                           class_weight='balanced'
                          )

# Model Fit anf Preditions
clf.fit(train_x,train_y)
pred_y=clf.predict(test_x)

#Import scikit-learn metrics, classification_report and accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Model Accuracy and Classification Report
print("Accuracy:",metrics.accuracy_score(test_y, pred_y))
print(classification_report(test_y, pred_y,digits=2))

Accuracy: 0.924061433447099
              precision    recall  f1-score   support

           1       0.91      0.93      0.92       924
           2       0.94      0.95      0.94      1323
           3       0.87      0.67      0.75        39
           7       0.89      0.50      0.64        50
           8       1.00      0.38      0.55         8

    accuracy                           0.92      2344
   macro avg       0.92      0.68      0.76      2344
weighted avg       0.92      0.92      0.92      2344



#### Confusion Matrix

In [122]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(test_y, pred_y)
matrix.diagonal()/matrix.sum(axis=1)
matrix

array([[ 857,   64,    0,    3,    0],
       [  64, 1255,    4,    0,    0],
       [   3,   10,   26,    0,    0],
       [  15,   10,    0,   25,    0],
       [   2,    3,    0,    0,    3]], dtype=int64)

#### Combine DataFrame with predictions

In [343]:
# Reset Index
test_x0 = test_x0.reset_index().drop('index', 1)
test_y = test_y.reset_index().drop('index', 1)

# Combine DataFrame with predictions
from pandas import DataFrame
t = DataFrame(pred_y, columns=['Prediction'])
table = pd.concat([test_x0, test_y, t], axis=1)
table[:3]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction
0,Woolworth,continental cup a soup,mild chicken curry with lots of noodles,Pantry,209.0,1.0,1.00,0.5,8.8,0.7,275.0,2,2
1,Coles,SPC,Spaghetti with Cheese 4 pack,Pantry,304.0,1.8,0.25,0.2,14.6,4.4,397.0,1,1
2,Coles,Allen's,Fantales Family Size,Pantry,1880.0,2.9,15.80,13.4,73.0,56.0,120.0,2,2


#### True/False share

In [ ]:
t = table[['ADG', 'Prediction']]
t["Result"] = t.drop("ADG", 1).isin(t["ADG"]).any(1)

In [362]:
t['Result'].value_counts(normalize=True) * 100

True     92.406143
False     7.593857
Name: Result, dtype: float64

#### Add True/False (Correct/Incorrect Prediction) Column 

In [345]:
data = pd.concat([test_x0, t], axis=1)
data

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
0,Woolworth,continental cup a soup,mild chicken curry with lots of noodles,Pantry,209.0,1.000000,1.000000,0.50,8.800000,0.700000,275.0,2,2,True
1,Coles,SPC,Spaghetti with Cheese 4 pack,Pantry,304.0,1.800000,0.250000,0.20,14.600000,4.400000,397.0,1,1,True
2,Coles,Allen's,Fantales Family Size,Pantry,1880.0,2.900000,15.800000,13.40,73.000000,56.000000,120.0,2,2,True
3,Woolworth,infuzions,thai sweet chilli potato mix,Pantry,1960.0,3.900000,9.500000,2.10,71.000000,8.600000,904.0,2,1,False
4,Woolworth,buderim,ginger glace,Pantry,1349.0,0.500000,0.250000,0.05,78.599998,72.500000,23.0,2,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,Coles,Hoyts,Bouquet Garni 4 Sachets,Pantry,1217.0,11.600000,9.900000,0.00,60.700001,0.000000,44.0,3,1,False
2340,Woolworth,cadbury,curly wurly,Pantry,1920.0,3.100000,17.799999,9.60,70.800003,49.099998,234.0,2,2,True
2341,Coles,Carman's,Gluten Free Salted Dark Choc & Almond Bars 5 pack,Pantry,2140.0,27.700001,29.100000,6.40,30.299999,17.000000,174.0,2,2,True
2342,Coles,Planet Food,Organic Almond Flour,Pantry,2440.0,23.500000,47.299999,3.70,13.000000,5.000000,2.0,1,1,True


#### Miscassifications for every class

In [351]:
false = data.sort_values(by=['Result'])[:178]

one = false.sort_values(by=['ADG'])[:67]
two = false.sort_values(by=['ADG'])[67:135]
three = false.sort_values(by=['ADG'])[135:148]
seven = false.sort_values(by=['ADG'])[148:173]
eight = false.sort_values(by=['ADG'])[173:]

#### Class 1

In [352]:
one[:2]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
7,Coles,Tassal,Premium Tasmanian Roasted Salmon Asian Style S...,Pantry,904.0,12.8,15.1,1.9,7.5,6.3,320.0,1,2,False
799,Coles,Golden Circle,100% Apple Juice No Added Sugar,Drinks,205.0,0.1,0.0,0.0,11.6,10.9,5.0,1,2,False


#### Class 2

In [353]:
two[:2]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
1250,Coles,Golden Circle,Apricot Nectar Fruit Drink,Drinks,270.0,0.2,0.0,0.0,15.100000,14.3,2.0,2,1,False
1544,Woolworth,the spice tailor,hyderabad biryani,Pantry,532.0,2.8,2.1,1.0,22.200001,0.8,173.0,2,1,False


#### Class 3

In [354]:
three[:2]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
1344,Woolworth,hoyts,italian herb mix,Pantry,2270.0,22.0,10.4,1.8,104.000000,50.599998,121.0,3,2,False
2339,Coles,Hoyts,Bouquet Garni 4 Sachets,Pantry,1217.0,11.6,9.9,0.0,60.700001,0.000000,44.0,3,1,False


#### Class 7

In [355]:
seven[:2]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
185,Coles,Whole Kids,Organic Cheese & Leek Dinosaur Puffs,Baby,1782.0,9.100000,16.5,3.1,58.700001,1.8,72.0,7,1,False
2046,Coles,Rafferty's Garden,Banana Milk Teething Rusks,Baby,1480.0,17.299999,0.3,0.1,66.599998,8.1,330.0,7,1,False


#### Class 8

In [356]:
eight[:2]

,Source,Brand,Product name,Category,Energy,Protein,Fat,Sat_Fat,Carbo_Hydrate,Sugar,Sodium,ADG,Prediction,Result
785,Woolworth,kellogg's,variety pack cereals,Pantry,1518.0,10.1,1.16,0.25,72.230003,32.650002,335.0,8,1,False
1285,Woolworth,marion's kitchen,thai massaman curry cooking kit,Pantry,1240.0,7.5,23.90,11.40,13.600000,6.600000,1500.0,8,2,False


#### Count Number of test samples for every class

In [360]:
sum1 = test_y['ADG'].value_counts()[1]
sum2 = test_y['ADG'].value_counts()[2]
sum3 = test_y['ADG'].value_counts()[3]
sum7 = test_y['ADG'].value_counts()[7]
sum8 = test_y['ADG'].value_counts()[8]
sum1

924

#### The percentage of misclassified data point by class

In [361]:
print('Class 1:', round(len(one)/sum1*100, 2), '%')
print('Class 2:', round(len(two)/sum2*100, 2), '%')
print('Class 3:', round(len(three)/sum3*100, 2), '%')
print('Class 7:', round(len(seven)/sum7*100, 2), '%')
print('Class 8:', round(len(eight)/sum8*100, 2), '%')

Class 1: 7.25 %
Class 2: 5.14 %
Class 3: 33.33 %
Class 7: 50.0 %
Class 8: 62.5 %
